# **Práctica 1**

En esta práctica aprenderemos como entrenar una red neuronal que nos prediga el precio que va a tener un teléfono móvil a partir de un [histórico de datos](https://www.kaggle.com/iabhishekofficial/mobile-price-classification). En esta sesión aprenderemos:

1.   Cargar los datos.
2.   Preprocesar los datos.
3.   Definir una architectura de red.
4.   Seleccionar la función de coste y el optimizador.
5.   Entrenar la red.



# 1. Cargado de los datos

In [1]:
ls ./data

test.csv  train.csv


In [2]:
# Importamos dependencias
import numpy as np
import pandas as pd

In [3]:
# Cargamos el conjunto de datos de entrenamiento
datos = pd.read_csv('./data/train.csv')

In [ ]:
# Visualizamos tamaño
# 2000 muestras de entrenamiento
# 20 variables + 1 objetivo (price_range)
datos.shape

In [ ]:
# Visualizamos las variables
datos.columns

In [ ]:
# Mostramos las primeras 10 muestras
datos.head(10)

# 2. Preprocesado de los datos

In [7]:
# Importamos dependencias preprocesado
from sklearn.preprocessing import StandardScaler # Normalización datos para que estén en la misma escala
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split

In [ ]:
# Diferenciamos variables entrada de variable de salida
salida = datos['price_range']
datos.pop('price_range')

In [ ]:
# Vemos tamaños de los datos
print('Tamaño datos entrada: ', datos.shape)
print('Tamaño variable salida: ', salida.shape)

In [ ]:
# Convertimos DataFrame a Numpy
X = datos.values
y = salida.values
print(y.shape)
y = np.reshape(y, (y.shape[0], 1))
print(y.shape)

In [ ]:
# Vemos cómo se han convertido los formatos de datos y qué tamaño tienen 
# nuestros datos
print('Tipo dato anterior: ', type(datos))
print('Tipo dato nuevo: ', type(X))

In [12]:
# Normalizamos los datos de entrada (z-score)
sc = StandardScaler()
sc.fit(X)
X_norm = sc.transform(X)

In [ ]:
# Vemos los datos antes y después de la normalización
print("Battery power antes de la normalización: ")
print(X[0,:])
print("Battery power después de la normalización: ")
print(X_norm[0,:])

In [ ]:
# Vemos que tras la normalización las variables están en el mismo rango y 
# evitamos que unas tengan más peso que otras.
print("Battery power antes de la normalización: ")
print(X[:,0])
print("Battery power después de la normalización: ")
print(X_norm[:,0])

print("Clock speed antes de la normalización: ")
print(X[:,2])
print("Clock speed después de la normalización: ")
print(X_norm[:,2])

In [ ]:
# Codificación one-hot de la variable de salida
print("Valores variable de salida: ", np.unique(y))
onehot_enc = OneHotEncoder()
y_onehot = onehot_enc.fit_transform(y).toarray()
print("Valor antes de la codificación: ", y[0])
print("Valor después de la codificación: ", y_onehot[0])

In [16]:
# Dividimos nuestro conjunto de datos en entrenamiento y validación
X_train, X_testval, y_train, y_testval = train_test_split(X_norm, y_onehot, 
                                                          test_size=0.2)

In [ ]:
# Vemos el tamaño de las particiones
print("Tamaño datos de entrada entrenamiento: ", X_train.shape)
print("Tamaño salida entrenamiento: ", y_train.shape)
print("Tamaño datos de entrada validación/test: ", X_testval.shape)
print("Tamaño salida validación/test: ", y_testval.shape)

In [18]:
# Dividimos nuestro conjunto de datos de validacion en test y validación
X_val, X_test, y_val, y_test = train_test_split(X_testval, y_testval, 
                                                  test_size=0.5)

In [ ]:
# Vemos el tamaño de las nuevas particiones
print("Tamaño datos de entrada validación: ", X_val.shape)
print("Tamaño salida validación: ", y_val.shape)
print("Tamaño datos de entrada test: ", X_test.shape)
print("Tamaño salida test: ", y_test.shape)

# 3. Definición de la arquitectura de la Red Neuronal

In [20]:
# Importamos dependencias
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense # Capa totalmente conectada

In [ ]:
# Definimos arquitectura
model = Sequential()
# Capa de entrada: X_train.shape[1], numero de variables
model.add(Input(shape=(X_train.shape[1],)))
# Capas ocultas. 2 capas ocultas con 16 y 12 nodos respectivamente
model.add(Dense(16, activation='relu'))
model.add(Dense(12, activation='relu'))
# Capa de salida. 4 clases - Clasificación multiclase
model.add(Dense(4, activation='softmax'))
model.summary()

Número de parámetros a ajustar por capa: **número_entrada * número_neuronas**. Ejemplo:

dense_1 = (n_entradas + 1) * n_neuronas = (20 + 1) * 16 = 336

(+1 del bias)



# 4. Definición de la función de pérdidas y el optimizador

In [22]:
# Función de pérdidas: clasificación -> entropía cruzada.
# Optimizador: tasa de aprendizaje adaptativa -> optimizador Adam.
# Métricas: estadísticos que queremos que se calculen tras cada iteración 
# para evaluar el rendimiento de la red.
model.compile(loss="categorical_crossentropy", optimizer="adam",
              metrics=["accuracy"])

# 5. Entrenamiento

In [ ]:
history = model.fit(X_train, y_train, epochs=100, batch_size=64,
                    validation_data=(X_val, y_val))

Cosas a observar:

*   Métricas entrenamiento
*   Métricas validacón
*   ¿Sobreajuste?



In [ ]:
# Visualizamos la precisión
import matplotlib.pyplot as plt
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Precisión modelo')
plt.ylabel('Precisión')
plt.xlabel('Época')
plt.legend(['Entrenamiento', 'Validación'], loc="lower right")
plt.show()

In [ ]:
# Visualizamos pérdidas
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Pérdidas modelo')
plt.ylabel('Pérdidas')
plt.xlabel('Época')
plt.legend(['Entrenamiento', 'Validación'], loc="upper right")
plt.show()

In [26]:
# Guardamos el modelo
from pathlib import Path
path_modelos = Path('./modelos')
path_modelos.mkdir(exist_ok=True)
model.save(path_modelos / 'model_1.h5')

In [ ]:
# Evaluamos el modelo
metrics_evaluation = model.evaluate(X_test, y_test, verbose=0)
print('Precisión test: ', metrics_evaluation[1])

# 6. Uso del modelo entrenado sobre los datos de test

Mediante el conjunto de datos de entrenamiento/validación hacemos el entrenamiento de la red y la selección de los hiperparámetros óptimos. Sin embargo, para conocer el rendimiento real del modelo entrenado debemos evaluarlo empleando un nuevo conjunto de datos, los datos de test.


In [28]:
# Cargamos los datos de test
data_test = pd.read_csv('./data/test.csv')

In [ ]:
# Mostramos tamaño datos y nombre variables
print(data_test.shape)
print(data_test.columns)

ATENCIÓN: Variable ID no se encuentra en el set de entrenamiento. Debemos eliminarla

In [ ]:
# Primeras filas
data_test.head()

In [ ]:
# Eliminamos columna id
data_test.pop('id')
print(data_test.columns)

In [ ]:
# Convertimos DataFrame a Numpy
data_test = data_test.values
print(type(data_test))

In [ ]:
# Normalizamos datos de test. 
# IMPORTANTE: Normalizamos datos con media y std aprendidas del set de entrenamiento.
data_norm = sc.transform(data_test)
print('Datos antes de normalizar: ')
print(data_test)
print('Datos después de normalizar')
print(data_norm)

In [34]:
# Cargamos modelo generado anteriormente
from tensorflow.keras.models import load_model
model = load_model('./modelos/model_1.h5')

In [35]:
# Realizamos predicciones
predicciones = model.predict(data_norm)

In [ ]:
# Visualizamos predicciones
print(predicciones)

In [37]:
# Convertimos probabilidad en clase final. Nos quedamos con la clase de mayor probabilidad
predicciones_clase = np.argmax(predicciones, axis=1)

In [ ]:
# Visualizamos predicciones
print(predicciones_clase)

# Ejercicio 1. Entrenamiento sin normalizar los datos

# Ejercicio 2: Arquitectura menos compleja
En este ejercicio vamos a comprobar como se comporta el modo si reducimos la complejidad del modelo. Esto se puede realizar de dos formas distintas:


1.   Reduciendo el número de capas ocultas.
2.   Reduciendo el número de neuronas por capa.



# Ejercicio 3: Arquitectura más compleja
En este ejercicio vamos a ver cómo afecta al rendimiento de la red aumentar la complejidad de la red. Esto se puede realizar de dos formas distintas:


1.   Aumentando el número de capas ocultas.
2.   Aumentando el número de neuronas por capa.



# Ejercicio 4: Tasa de aprendizaje
En todos los ejemplos hasta ahora hemos empleado una tasa de aprendizaje variable a través del optimizador Adam. En este ejercicio vamos a evaluar el rendimiento de la red empleando distintas tasas de aprendizaje.


1.   Tasa de aprendizaje constante elevada.
2.   Tasa de aprendizaje constante pequeña.
3.   Planificador de tasa de aprendizaje (RMSProp, Adam, Adagrad y Adadelta).

NOTA: Para la tasa de aprendizaje constante se empleará el optimizador de descenso de gradiente estocástico (SGD).



# Ejercicio 5: EarlyStopping

Para emplear la técnica de reularización de EarlyStopping podemos hacer uso de los callbacks de Keras. Estas son funciones que se ejecutan en diferentes etapas del proceso de entrenamiento (e.g. al principio o final de cada época, antes o después de un batch, etc.). El callback de EarlyStopping detiene el entrenamiento cuando una determinada métrica deja de mejorar.


# Ejercicio 6: Dropout

Introduce Dropout en las capas ocultas.

# Ejercicio 7: Regularización L1

Introduce regularización L1 a las capas ocultas.

# Ejercicio 8: Regularización L2
Introduce regularización L2 a las capas ocultas.

# Extra: Selección de hiperparámetros (grid search)
Como hemos visto tanto en la sesión teórica como en la sesión práctica, hay varios hiperparámetros que el desarrollador debe definir a la hora de diseñar la arquitectura. 
Esta búsqueda se basa en la experimentación (prueba y error) de forma que consigamos la configuración que mejor resultados nos de sobre el conjunto de datos de validación.
Existen métodos que facilitan esta búsqueda como "GridSearchCV" de sckit-learn. En este definimos qué parametros queremos probar y, para cada uno de ellos, con qué valores. Una vez definidos, siguiendo una estrategía de validación cruzada, selecciona la mejor configuración.

Pese a ser una buena técnica para buscar hiperparámetros se recomienda acotar lo máximo posible las pruebas a realizar ya que, dependiendo de la complejidad de la arquitectura y el tamaño del dataset, puede resultar computacionalmente costoso.

In [ ]:
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier

# Método para crear el modelo.
def create_model(optimizer):
  model = Sequential()
  model.add(Input(shape=(X_train.shape[1],)))
  model.add(Dense(16, activation='relu'))
  model.add(Dense(12, activation='relu'))
  model.add(Dense(4, activation='softmax'))
  model.compile(loss="categorical_crossentropy", optimizer=optimizer,
              metrics=["accuracy"])
  return model

# Define los parámetros a probar
batch_size = [20, 40, 60, 80, 100]
epochs = [10, 50, 100, 150]
optimizer = ['SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Adam']
param_grid = dict(optimizer=optimizer, batch_size=batch_size, epochs=epochs)

# Creamos modelo
model = KerasClassifier(build_fn=create_model, verbose=0)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X_norm, y_onehot)

# Resultados
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.923000 using {'batch_size': 20, 'epochs': 150, 'optimizer': 'SGD'}
0.630002 (0.049033) with: {'batch_size': 20, 'epochs': 10, 'optimizer': 'SGD'}
0.820494 (0.014574) with: {'batch_size': 20, 'epochs': 10, 'optimizer': 'RMSprop'}
0.259995 (0.015461) with: {'batch_size': 20, 'epochs': 10, 'optimizer': 'Adagrad'}
0.257997 (0.016424) with: {'batch_size': 20, 'epochs': 10, 'optimizer': 'Adadelta'}
0.796981 (0.029291) with: {'batch_size': 20, 'epochs': 10, 'optimizer': 'Adam'}
0.907990 (0.014367) with: {'batch_size': 20, 'epochs': 50, 'optimizer': 'SGD'}
0.916497 (0.006075) with: {'batch_size': 20, 'epochs': 50, 'optimizer': 'RMSprop'}
0.315999 (0.001662) with: {'batch_size': 20, 'epochs': 50, 'optimizer': 'Adagrad'}
0.228503 (0.038773) with: {'batch_size': 20, 'epochs': 50, 'optimizer': 'Adadelta'}
0.913005 (0.010843) with: {'batch_size': 20, 'epochs': 50, 'optimizer': 'Adam'}
0.912995 (0.006544) with: {'batch_size': 20, 'epochs': 100, 'optimizer': 'SGD'}
0.900499 (0.006979) with: {